## チュートリアル説明

本チュートリアルでは、最終的にニュースコーパスから特徴量を抽出することを目的とする。  
各章では、特徴量を抽出するにあたって、使用しているデータがどのような特性を持つかを解析し、必要に応じて正規化し、可視化する方法を説明する。  
最後の章では、自然語処理モデルであるBERT(Bidirectional Encoder Representations from Transformers)を用いて、特徴量を抽出する方法を説明する。また、抽出された特徴量をLSTMを用いて週ごとに合成する方法を説明する。

## 1.3. 基本準備
本章では、必要なライブラリーと分析用辞書などをインストールし、チュートリアルのコードを正しく利用できる環境を構築する。  
また、本チュートリアルで利用するライブラリーを宣言し、その他、チュートリアルを進めるにあたって必要な基本的な準備を行う。

In [ ]:
# Google Colab環境ではGoogle Driveをマウントしてアクセスできるようにします。
import sys

if 'google.colab' in sys.modules:
    # Google Drive をマウントします
    from google.colab import drive
    mount_dir = "/content/drive"
    drive.mount(mount_dir)

### 1.3.3. 必要なライブラリのインストール

In [ ]:
!apt update
!apt install -y build-essential sudo mecab libmecab-dev mecab-ipadic-utf8 fonts-ipafont-gothic file
!pip install pandas==1.1.5 numpy==1.19.5 scattertext==0.1.0.0 wordcloud==1.8.1 torch==1.7.1 torchvision==0.8.2 transformers==4.2.2 mecab-python3==0.996.6rc1 ipadic==1.0.0 neologdn==0.4 fugashi==1.0.5 japanize-matplotlib==1.1.3 gensim==3.8.3 pyLDAvis==2.1.2

# mecab用の辞書をインストール
!git clone https://github.com/neologd/mecab-ipadic-neologd.git --branch v0.0.7 --single-branch
!yes yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd

### 1.3.4. ライブラリの読み込み

In [ ]:
# 基本ライブラリー
import re
import os
import sys
import math
import random
import json
import joblib
import numpy as np
import pandas as pd
from scipy import stats
import string
from copy import copy
from glob import glob
from itertools import chain
import gc

# テキスト解析関連
import MeCab
import unicodedata
import neologdn

# 可視化関連
from tqdm.auto import tqdm
from IPython.display import display, display_markdown, IFrame
import scattertext as st
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
import gensim
import pyLDAvis
import pyLDAvis.gensim

# ニューラルネット関連
import torch
from torch import nn
import torch.nn.functional as F
import transformers
from transformers import BertJapaneseTokenizer
from torch.utils.data import DataLoader, Dataset as _Dataset

# ノートブック上でpyLDAvisより可視化を行う場合の設定
pyLDAvis.enable_notebook()

### 1.3.6. 実行環境の確認

In [ ]:
print(sys.version)

### 1.3.7. ファイルパスの設定

In [ ]:
# colab環境で実行する場合
if 'google.colab' in sys.modules:
    CONFIG = {
        'base_path': f'{mount_dir}/MyDrive/JPX_competition/workspace',
        'article_path': f'{mount_dir}/MyDrive/JPX_competition/data_dir_comp2/nikkei_article.csv.gz',
        'stock_price_path': f'{mount_dir}/MyDrive/JPX_competition/data_dir_comp2/stock_price.csv.gz',
        'stock_list_path': f'{mount_dir}/MyDrive/JPX_competition/data_dir_comp2/stock_list.csv.gz',
        'dict_path': '/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd',
        'font_path': '/usr/share/fonts/truetype/fonts-japanese-gothic.ttf',
    }
else:
    CONFIG = {
        'base_path': '/notebook/workspace',
        'article_path': '/notebook/data_dir_comp2/nikkei_article.csv.gz',
        'stock_price_path': '/notebook/data_dir_comp2/stock_price.csv.gz',
        'stock_list_path': '/notebook/data_dir_comp2/stock_list.csv.gz',
        'dict_path': '/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd',
        'font_path': '/usr/share/fonts/truetype/fonts-japanese-gothic.ttf',
    }

# 1.3.10. 本番提出用のクラス作成

In [ ]:
# 本番提出用のクラスを作成するため、関数の持たない基本クラスを定義する
# 下記で段階的にクラスを作り上げる
class SentimentGenerator(object):
    # 以下は使用時にビルドされる。
    # 各々に関しては、以下のチュートリアルで説明する。
    article_columns = None
    punctuation_replace_dict = None
    punctuation_remove_list = None
    device = None
    feature_extractor = None
    headline_feature_combiner_handler = None
    keywords_feature_combiner_handler = None

## 1.7.4. LSTMによる可変特徴量統合
本チュートリアルの最終モデルの評価関数は、Marketのweekly return予測することとして判定される。週ごとにデータ数が動的であるニュースデータは、それらから抽出した特徴量もその数が可変であり、それらの可変する情報を用いてモデリングする必要がある。本チュートリアルでは、週ごとのその数が可変する特徴量を一つの統合する戦略をとっている。そのため、可変長さの入力が可能なLSTMを用いて、Market returnの次の週の上げ下げの情報をラベルとして、特徴量を統合する学習を行った。

本章では、そのLSTMのモデル作成及び、学習の方法を説明し、最終的に単一特徴量として抽出する方法を説明する。

### 1.7.5. BERT特徴量ロード

In [ ]:
headline_features = pd.read_pickle(f'{CONFIG["base_path"]}/headline_features/headline_features.pkl')
keywords_features = pd.read_pickle(f'{CONFIG["base_path"]}/keywords_features/keywords_features.pkl')

In [ ]:
headline_features.head(3)

In [ ]:
keywords_features.head(3)

### Stockデータロード

In [ ]:
# stock priceとstock listをロードする。
stock_price = pd.read_csv(CONFIG["stock_price_path"])
stock_list = pd.read_csv(CONFIG["stock_list_path"])

In [ ]:
stock_price.head(3)

In [ ]:
stock_list.head(3)

In [ ]:
# stock_priceのデータ精製を行う。
# ラベル作成のために使用するcolumnは['EndOfDayQuote Date', 'Local Code', "EndOfDayQuote Open", "EndOfDayQuote ExchangeOfficialClose"]のみである。
# EndOfDayQuote Date: データの日付
# Local Code: 銘柄コード
# EndOfDayQuote Open: 始値
# EndOfDayQuote ExchangeOfficialClose: 終値
stock_price = stock_price[['EndOfDayQuote Date', 'Local Code', "EndOfDayQuote Open", "EndOfDayQuote ExchangeOfficialClose"]]

# それぞれのcolumn名をわかりやすく変更する
stock_price = stock_price.rename(columns={
    'EndOfDayQuote Date': 'date',
    'Local Code': 'asset',
    'EndOfDayQuote Open': 'open',
    'EndOfDayQuote ExchangeOfficialClose': 'close',
})

# データごとにindex形式が異なると大変扱いにくい。下記のコードより特徴量と同様のindexの形式を変更する。
# pd.to_datetimeより、string形式の日付をpd.Timestamp形式に変換する
# pd.Timestamp形式をpd.DatetimeIndex形式に変更し、time zoneをheadline_featuresと同様に設定する。
# この際、headline_featuresとkeywords_featuresはarticlesのindexを使用しているため、timezoneが一致している。どちらを用いても良い。
stock_price['date'] = pd.to_datetime(stock_price['date'])
stock_price['date'] = pd.DatetimeIndex(stock_price['date']).tz_localize(headline_features.index.tz)

# indexを['date', 'asset']順のpd.MultiIndex形式として設定する。
stock_price = stock_price.set_index(['date', 'asset']).sort_index()

# unstack()より銘柄情報をcolumnに移動させる。
stock_price = stock_price.unstack()

# 今回使用するデータは2020年以降のデータであるので、2020年以前のデータを切り捨てる。
stock_price = stock_price['2020-01-01':]

# 確認する
display(stock_price.head())

In [ ]:
# stock_listデータから2020年12月末日時点での株式発行数の情報のみを取得する。
# 使用するcolumnは、['Local Code', 'IssuedShareEquityQuote IssuedShare']である。
# Local code: 銘柄コード
# IssuedShareEquityQuote IssuedShare: 発行株式数
stock_list = stock_list[['Local Code', 'IssuedShareEquityQuote IssuedShare']]

# columns名をわかりやすく変更する
stock_list = stock_list.rename(
    columns={
        'Local Code': 'asset',
        'IssuedShareEquityQuote IssuedShare': 'shares'
    }
)

# assetをindexとして設定する
shares = stock_list.set_index('asset')['shares']

# 確認する
display(shares)

### 使用する銘柄選定

本章では、以下の条件に従い、マーケットのリターンを計算する際に用いる銘柄を選定する。

1. 2020年12月末日時点で、東京証券取引所に上場していること。
2. 2020年12月末日時点で、時価総額が200億を上回っていること。

上の条件を満たす銘柄を選ぶため、以下のロジックに従う。
1. 12月末日時点で価格情報が存在するものを上場しているとみなす。
2. 12月末日時点での株式発行数 * 終値から時価総額を計算し、200億円未満のものを切り捨てる。

In [ ]:
# 条件１「2020年12月末日時点で、東京証券取引所に上場していること」に該当する銘柄を取得する。
# まず、indexの最終日を取得する。
last_date = stock_price.index[-1]

# 2020年の末日が、最終日であることがわかる。
display(last_date)

In [ ]:
# 2020年の末日時点での終値が存在する銘柄を取得する。
universe_condition_1 = stock_price.xs(last_date)["close"].dropna().index

# 条件一に該当する銘柄数を確認する。
print(f'number_of_assets: {len(universe_condition_1)}/{len(stock_price["close"].columns)}')

In [ ]:
# 条件２「2020年12月末日時点で、時価総額が200億を上回っていること」に該当する銘柄を取得する。
# まず、株式発行数 * 終値から時価総額を計算する。(単位: 円)
marketcap = (stock_price.xs(last_date)['close'] * shares)

# 確認する
display(marketcap)

In [ ]:
# 時価総額が200億円未満のものを切り捨てる
universe_condition_2 = marketcap[marketcap >= 20000000000].index

# 条件一に該当する銘柄数を確認する。
print(f'number_of_assets: {len(universe_condition_2)}/{len(stock_price["close"].columns)}')

In [ ]:
# 上の二つの条件を合成し、universeを設定する。
universe = universe_condition_1 & universe_condition_2

# universeをcsvでstoreしておきます。
universe.to_series().rename('universe').to_frame().to_csv('universe.csv')

# 条件に該当する銘柄数を確認する。
print(f'number_of_assets: {len(universe)}/{len(stock_price["close"].columns)}')

In [ ]:
# 選定された銘柄のみの価格情報に精製する。
stock_price = stock_price[[column for column in stock_price.columns if column[-1] in universe]]

### 1.7.6. 週ごとにグループされた特徴量とラベルの作成

In [ ]:
# 各週ごとの特徴量を統合するためには、週ごとの全ての特徴量をグループすると扱いやすくなる。
# ここでは、週ごとに特徴量とプライス情報をグループする方法を説明する。
# また、週ごとのプライス情報をグループし、weekly returnを計算し、ラベルを作成する方法を説明する。
@classmethod
def _build_weekly_group(cls, df):
    # index情報から、(year, week)の情報を得る。
    return pd.Series(list(zip(df.index.year, df.index.week)), index=df.index)

# SentimentGeneratorに定義したclassmethodを追加する
SentimentGenerator._build_weekly_group = _build_weekly_group

# 特徴量に適用してみる
display_markdown('#### features', raw=True)
features = headline_features
weekly_group = SentimentGenerator._build_weekly_group(df=features)
display(weekly_group.head(3))
display(weekly_group.tail(3))

# プライスに適用してみる。
# stock priceの2020年の1週目は、データが存在しないため、2020年の２週目から存在することがわかる。
# これらのindexをマッチさせることは後ほど説明する。
display_markdown('#### stock price', raw=True)
weekly_group = SentimentGenerator._build_weekly_group(stock_price)
display(weekly_group.head(3))
display(weekly_group.tail(3))

In [ ]:
# 特徴量を週ごとにグループ化してみる
weekly_group = SentimentGenerator._build_weekly_group(df=features)
features = features.groupby(weekly_group).apply(lambda x: x[:])

# 週の情報がindexのlevel 0に付与され、グループされていることがわかる。
features.head(3)

In [ ]:
# trainとtestを区切る週をboundary_weekとして設定し、train用に用いられる特徴量と、test用に用いられる特徴量を区切る。
boundary_week = (2020, 26)
train_features = features[features.index.get_level_values(0) <= boundary_week]
test_features = features[features.index.get_level_values(0) > boundary_week]

display_markdown('#### train_features', raw=True)
display(train_features.head(3))
display(train_features.tail(3))

display_markdown('#### test_features', raw=True)
display(test_features.head(3))
display(test_features.tail(3))

In [ ]:
# 今回は、stock priceを週ごとにグループし、翌週のopen to closeのreturn、つまりweekly forward returnをビルドする方法を説明する。
# weekly forward returnをビルドする前に、まず、その週のopen to closeの weekly returnをビルドしてみよう。
# weekly returnをビルドする関数を定義する。
def _compute_weekly_return(x):
    # その週の初営業日のopenから最終営業日のcloseまでのリターンを計算する。
    weekly_return = ((x['close'].iloc[-1] - x['open'].iloc[0]) / x['open'].iloc[0])
    
    # その日のvolumneが0であるデータは、openが0となっている。
    # openが0の場合、np.infの値となっているため、np.nanに変換し除去する。
    # 銘柄ごとのリターンを単純平均し、marketのweekly_returnを計算する。
    return weekly_return.replace([np.inf, -np.inf], np.nan).dropna().mean()

weekly_group = SentimentGenerator._build_weekly_group(df=stock_price)
weekly_return = stock_price.groupby(weekly_group).apply(_compute_weekly_return)

display(weekly_return.head(3))
display(weekly_return.tail(3))

In [ ]:
# weekly_returnをshift(-1)することより、翌週のreturn情報が現在のindexに入るようになる。
weekly_fwd_return = weekly_return.shift(-1).dropna()

display(weekly_fwd_return.head(3))
display(weekly_fwd_return.tail(3))

In [ ]:
# trainとtestを区切る週をboundary_weekとして設定し、train用に用いられる特徴量と、test用に用いられる特徴量を区切る。
boundary_week = (2020, 26)
train_labels = weekly_fwd_return[weekly_fwd_return.index <= boundary_week]
test_labels = weekly_fwd_return[weekly_fwd_return.index > boundary_week]

display_markdown('#### train_labels', raw=True)
display(train_labels.head(3))
display(train_labels.tail(3))

display_markdown('#### test_labels', raw=True)
display(test_labels.head(3))
display(test_labels.tail(3))

In [ ]:
# fwd_returnの上げ下げの情報のみをラベルとして用いるため、上げを1.0, 下げを0.0に変換する
train_labels = (train_labels >= 0) * 1.0
test_labels = (test_labels >= 0) * 1.0

display_markdown('#### train_labels', raw=True)
display(train_labels.head(3))
display(train_labels.tail(3))

display_markdown('#### test_labels', raw=True)
display(test_labels.head(3))
display(test_labels.tail(3))

In [ ]:
# 上記のコードを関数としておとす。
@classmethod
def build_weekly_features(cls, features, boundary_week):
    assert isinstance(boundary_week, tuple)
    
    weekly_group = cls._build_weekly_group(df=features)
    features = features.groupby(weekly_group).apply(lambda x: x[:])

    train_features = features[features.index.get_level_values(0) <= boundary_week]
    test_features = features[features.index.get_level_values(0) > boundary_week]
    
    return {'train': train_features, 'test': test_features}

@classmethod
def build_weekly_labels(cls, stock_price, boundary_week):
    def _compute_weekly_return(x):
        # その週の初営業日のopenから最終営業日のcloseまでのリターンを計算する。
        weekly_return = ((x['close'].iloc[-1] - x['open'].iloc[0]) / x['open'].iloc[0])

        # その日のvolumneが0であるデータは、openが0となっている。
        # openが0の場合、np.infの値となっているため、np.nanに変換し除去する。
        # 銘柄ごとのリターンを単純平均し、marketのweekly_returnを計算する。
        return weekly_return.replace([np.inf, -np.inf], np.nan).dropna().mean()

    assert isinstance(boundary_week, tuple)

    weekly_group = cls._build_weekly_group(df=stock_price)
    weekly_fwd_return = stock_price.groupby(weekly_group).apply(_compute_weekly_return).shift(-1).dropna()

    train_labels = weekly_fwd_return[weekly_fwd_return.index <= boundary_week]
    test_labels = weekly_fwd_return[weekly_fwd_return.index > boundary_week]
    
    train_labels = (train_labels >= 0) * 1.0
    test_labels = (test_labels >= 0) * 1.0
    
    return {'train': train_labels, 'test': test_labels}

# SentimentGeneratorに定義したclassmethodを追加する
SentimentGenerator.build_weekly_features = build_weekly_features
SentimentGenerator.build_weekly_labels = build_weekly_labels

### 1.7.7. Pytorch Dataset作成
pytorchのモデルを効率よく学習させるためには、Custom datasetと、それを用いたdataloaderを定義する必要がある。ここでは、学習に適合するDatasetを定義する方法を説明する。  
Datasetの`__getitem__`は、学習や、テスト時、Dataloaderを介してデータを取得するときに呼ばれ、`__init__`で事前に定義したデータをidからインデックシングする仕組みになっている。  
そのため、idが付与されたら、そのidからfeatureとlabelをインデックシングできるような構造で`__init__`でデータを事前に定義しておく必要がある。

In [ ]:
# ここでは、例として、trainデータのみを用いて、__init__で事前定義するデータをビルトしてみよう。
# 任意に特徴量としては、headlineを使い、boundary_weekに2020年の26週目を設定。
features = headline_features
boundary_week = (2020, 26)

# 上記で作成したコードを用いて週グループの特徴量とラベルをビルドする。
weekly_features = SentimentGenerator.build_weekly_features(features=features, boundary_week=boundary_week)['train']
weekly_labels = SentimentGenerator.build_weekly_labels(stock_price=stock_price, boundary_week=boundary_week)['train']

In [ ]:
# 共通する週のみのデータを使うため、共通するindex情報を取得する。
mask_index = weekly_features.index.get_level_values(0).unique() & weekly_labels.index
display(mask_index)

In [ ]:
# 共通するindexのみのデータだけでreindexを行う。
weekly_features = weekly_features[weekly_features.index.get_level_values(0).isin(mask_index)]
weekly_labels = weekly_labels.reindex(mask_index)

display_markdown('#### weekly_features', raw=True)
display(weekly_features.head(3))
display(weekly_features.tail(3))

display_markdown('#### weekly_labels', raw=True)
display(weekly_labels.head(3))
display(weekly_labels.tail(3))

In [ ]:
# idからweekの情報を取得できるよう、id_to_weekをビルドする
id_to_week = {id: week for id, week in enumerate(sorted(weekly_labels.index))}
id_to_week

In [ ]:
# 続き、__getitem__で付与されたidから、データを取得するロジックを説明する。
# idからweekの情報を取得する
# 例として、任意的にid = 10を用いる。
id = 10

week = id_to_week[id]
week

In [ ]:
# 学習時のリソース軽減のため、全ての特徴量を入力とするわけではなく、直近n個を入力とする。ここでは1000個として定義する。
max_sequence_length = 1000

x = weekly_features.xs(week, axis=0, level=0)[-max_sequence_length:]
y = weekly_labels[week]

# 上記のコードよりidが付与されたとき、idから週の情報を取得し、その週の情報から、特徴量とラベルを手にすることができた。
display_markdown('#### 特徴量', raw=True)
display(x.head(3))
print('shape:', x.shape)

display_markdown('#### ラベル', raw=True)
display(y)

In [ ]:
# pytorchでは、データをtorch.Tensorタイプとして扱うことが要求される。
# 以下で、np.ndarrayをtensor形式に変換することができる。
x = torch.tensor(x.values, dtype=torch.float)
y = torch.tensor(y, dtype=torch.float)

display(x)
display(y)

In [ ]:
# 今回、学習に用いるサンプル数が大変少ないため、過学習防止のため、少し工夫を重ねる。
# 全体的な特徴量(ニュースの情報)の順序は維持しつつ、入力とする特徴量を数分割し、その分割の中でシャッフルを行う方法である。
# この方法を用いることで、モデルが観察するデータが飛躍的に増加する効果が期待され、過学習を防止に繋がるはずである。
def _shuffle_by_local_split(x, split_size=50):
    return torch.cat([splitted[torch.randperm(splitted.size()[0])] for splitted in x.split(split_size, dim=0)], dim=0)

x = _shuffle_by_local_split(x=x)
display(x)

In [ ]:
# 学習中においては、データを一つずつロードし、学習を行うより、複数個を同時に(batchを組み)並列演算を行う方が時間短縮に繋がる。
# そのためには、行列を重ねる必要があり、データのシークエンスが同一である必要がある。本章では、このようなシークエンスの異なるデータを扱うため、
# max_sequence_lengthを決め、最大のsequenceを合わせ、また、sequenceがmax_sequence_lengthに達しない場合は、前から0を埋め、sequenceを合わせる戦略を取る。

# sequenceがmax_sequence_lengthに達しないrandomのtensorをxとして、定義し、以下のコードでzero paddingを行ってみる。
x = torch.randn(100, 768)
display_markdown('#### Padding前', raw=True)
display(x)
display(x.shape)

if x.size()[0] < max_sequence_length:
    x = F.pad(x, pad=(0, 0, max_sequence_length - x.size()[0], 0))
    
display_markdown('#### Padding後', raw=True)
display(x)
display(x.shape)

In [ ]:
# 上記のコードをまとめて、Datasetクラスを作成する。
class Dataset(_Dataset):
    def __init__(self, weekly_features, weekly_labels, max_sequence_length):
        # 共通する週のみを使うため、共通するindex情報を取得する
        mask_index = (
            weekly_features.index.get_level_values(0).unique() & weekly_labels.index
        )

        # 共通するindexのみのデータだけでreindexを行う。
        self.weekly_features = weekly_features[
            weekly_features.index.get_level_values(0).isin(mask_index)
        ]
        self.weekly_labels = weekly_labels.reindex(mask_index)

        # idからweekの情報を取得できるよう、id_to_weekをビルドする
        self.id_to_week = {
            id: week for id, week in enumerate(sorted(weekly_labels.index))
        }

        self.max_sequence_length = max_sequence_length

    def _shuffle_by_local_split(self, x, split_size=50):
        return torch.cat(
            [
                splitted[torch.randperm(splitted.size()[0])]
                for splitted in x.split(split_size, dim=0)
            ],
            dim=0,
        )

    def __len__(self):
        return len(self.weekly_labels)

    def __getitem__(self, id):
        # 付与されたidから週の情報を取得し、その週の情報から、特徴量とラベルを取得する。
        week = self.id_to_week[id]
        x = self.weekly_features.xs(week, axis=0, level=0)[-self.max_sequence_length :]
        y = self.weekly_labels[week]

        # pytorchでは、データをtorch.Tensorタイプとして扱うことが要求される。
        # 全体的な特徴量(ニュースの情報)の順序は維持しつつ、入力とする特徴量を数分割し、その分割の中でシャッフルを行う。
        x = self._shuffle_by_local_split(torch.tensor(x.values, dtype=torch.float))
        y = torch.tensor(y, dtype=torch.float)

        # max_sequence_lengthに最大のsequenceを合わせ、sequenceがmax_sequence_lengthに達しない場合は、前から0を埋め、sequenceを合わせる
        if x.size()[0] < self.max_sequence_length:
            x = F.pad(x, pad=(0, 0, self.max_sequence_length - x.size()[0], 0))

        return x, y

### 1.7.8. 特徴量合成モデル作成
本章では、特徴量統合モデルとして、LSTMを用いる。また、ラベルとして上げ下げのバイナリ情報を扱っているため、上げ下げの確率を出力とするモデルを定義し、binary_cross_entropyをlossとして学習を行う。出力層でのsentiment score及び、出力出力前層で、より高次元の情報を抽出し特徴量として用いる。

In [ ]:
class FeatureCombiner(nn.Module):
    def __init__(self, input_size, hidden_size, compress_dim=4, num_layers=2):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        # LSTMの定義、
        # batch_firstより、出力次元の最初がbatchとなる。
        # dropoutを用いて、内部状態のconnectionをdropすることより過学習を防ぐ。
        # Sequenceがかなり長いため、初期入力された情報の消失が激しいと想定される。それらの防止のため、bidirectionalのモデルを使う。
        self.cell = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.5,
            bidirectional=True,
        )

        # より高次元の特徴量を抽出できるようにするため、classifierの手前で、compress_dim次元への線形圧縮を行う。
        self.compressor = nn.Linear(hidden_size * 2, compress_dim)

        # sentiment probabilityの出力層。
        self.classifier = nn.Linear(compress_dim, 1)

        # outputの範囲を[0, 1]とする。
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # 入力値xから出力までの流れを定義する。
        output, _ = self.cell(x)
        output = self.sigmoid(self.classifier(self.compressor(output[:, -1, :])))
        return output

    def extract_feature(self, x):
        # 入力値xから特徴量抽出までの流れを定義する。
        output, _ = self.cell(x)
        output = self.compressor(output[:, -1, :])
        return output

### 1.7.9. 特徴量合成モデルのハンドラー作成
「3. Pytorch Dataset 作成, 4. 特徴量合成モデル作成」 において、データのインデクシングロジックや、特徴量合成モデルは作成することができた。しかし、これらだけではモデル学習は行えない。学習のロジックや、学習されたモデルのセーブとロード、推定、特徴量抽出を実装する必要がある。本章では、FeatureCombinerHandlerのクラスを定義し、そのようなロジックを実装する。

In [ ]:
class FeatureCombinerHandler:
    def __init__(self, feature_combiner_params, store_dir):
        # モデル学習及び推論に用いるデバイスを定義する
        if torch.cuda.device_count() >= 1:
            self.device = 'cuda'
            print("[+] Set Device: GPU")
        else:
            self.device = 'cpu'
            print("[+] Set Device: CPU")

        # モデルのcheckpointや抽出した特徴量及びsentimentをstoreする場所を定義する。
        self.store_dir = store_dir
        os.makedirs(store_dir, exist_ok=True)

        # 上記で作成したfeaturecombinerを定義する。
        self.feature_combiner = FeatureCombiner(**feature_combiner_params).to(
            self.device
        )

        # 学習に用いるoptimizerを定義する。
        self.optimizer = torch.optim.Adam(
            params=self.feature_combiner.parameters(), lr=0.001,
        )

        # ロス関数の定義
        self.criterion = nn.BCELoss().to(self.device)

        # モデルのcheck pointが存在する場合、モデルをロードする
        self._load_model()

    # 学習に必要なデータ(並列のためbatch化されたもの)をサンプルする。
    def _sample_xy(self, data_type):
        assert data_type in ("train", "val")

        # data_typeより、data_typeに合致したデータを取得するようにしている。
        if data_type == "train":
            # dataloaderをiteratorとして定義し、next関数として毎時のデータをサンプルすることができる。
            # Iteratorは全てのデータがサンプルされると、StopIterationのエラーを発するが、そのようなエラーが出たとき、
            # Iteratorを再定義し、データをサンプルするようにしている。
            try:
                x, y = next(self.iterable_train_dataloader)
            except StopIteration:
                self.iterable_train_dataloader = iter(self.train_dataloader)
                x, y = next(self.iterable_train_dataloader)

        elif data_type == "val":
            try:
                x, y = next(self.iterable_val_dataloader)
            except StopIteration:
                self.iterable_val_dataloader = iter(self.val_dataloader)
                x, y = next(self.iterable_val_dataloader)

        return x.to(self.device), y.to(self.device)

    # モデルのパラメータをアップデートするロジック
    def _update_params(self, loss):
        # ロスから、gradientを逆伝播し、パラメータをアップデートする
        loss.backward()
        self.optimizer.step()

    # 学習されたfeature_combinerのパラメータをcheck_pointとしてstoreするロジック
    def _save_model(self, epoch):
        torch.save(
            self.feature_combiner.state_dict(),
            os.path.join(self.store_dir, f"{epoch}.ckpt"),
        )
        print(f"[+] Epoch: {epoch}, Model is saved.")

    # 学習されたcheckpointが存在す場合、feature_combinerにそのパラメータをロードするロジック
    def _load_model(self):
        # cudaで学習されたモデルなどを、cpu環境下でロードするときはこのパラメータが必要となる。
        params_to_load = {}
        if self.device == "cpu":
            params_to_load["map_location"] = torch.device("cpu")

        # .ckptファイルを探し、古い順から新しい順にソートする。
        check_points = glob(os.path.join(self.store_dir, "*.ckpt"))
        check_points = sorted(
            check_points, key=lambda x: int(x.split("/")[-1].replace(".ckpt", "")),
        )

        # check_pointが存在しない場合は、スキップする。
        if len(check_points) == 0:
            print("[!] No exists checkpoint")
            return

        # 複数個のchieck_pointが存在する場合、一番最新のものを使い、モデルのパラメータをロードする
        check_point = check_points[-1]
        self.feature_combiner.load_state_dict(torch.load(check_point, **params_to_load))
        print("[+] Model is loaded")

    # Datasetからdataloaderを定義するロジック
    def _build_dataloader(
        self, dataloader_params, weekly_features, weekly_labels, max_sequence_length
    ):
        # 上記3で作成したしたdatasetを定義する
        dataset = Dataset(
            weekly_features=weekly_features,
            weekly_labels=weekly_labels,
            max_sequence_length=max_sequence_length,
        )

        # datasetのdataをiterableにロードできるよう、dataloaderを定義する、このとき、shuffle=Trueを渡すことで、データはランダムにサンプルされるようになる。
        return DataLoader(dataset=dataset, shuffle=True, **dataloader_params)

    # train用に、featuresとlabelsを渡し、datasetを定義し、dataloaderを定義するロジック
    def set_train_dataloader(
        self, dataloader_params, weekly_features, weekly_labels, max_sequence_length
    ):
        self.train_dataloader = self._build_dataloader(
            dataloader_params=dataloader_params,
            weekly_features=weekly_features,
            weekly_labels=weekly_labels,
            max_sequence_length=max_sequence_length,
        )

        # dataloaderからiteratorを定義する
        # iteratorはnext関数よりデータをサンプルすることが可能となる。
        self.iterable_train_dataloader = iter(self.train_dataloader)

    # validation用に、featuresとlabelsを渡し、datasetを定義し、dataloaderを定義するロジック
    def set_val_dataloader(
        self, dataloader_params, weekly_features, weekly_labels, max_sequence_length
    ):
        self.val_dataloader = self._build_dataloader(
            dataloader_params=dataloader_params,
            weekly_features=weekly_features,
            weekly_labels=weekly_labels,
            max_sequence_length=max_sequence_length,
        )

        # dataloaderからiteratorを定義する
        # iteratorはnext関数よりデータをサンプルすることが可能となる。
        self.iterable_val_dataloader = iter(self.val_dataloader)

    # 学習ロジック
    def train(self, n_epoch):
        # n_epochの回数分、全学習データを複数回用いて学習する。
        for epoch in range(n_epoch):

            # 各々のepochごとのaverage lossを表示するため、lossをstoreするリストを定義する。
            train_losses = []
            test_losses = []

            # train_dataloaderの長さは、全ての学習データを一度用いるときの長さと同様である。
            # batchを組むと、その分train_dataloaderの長さは可変し、ちょうど一度全てのデータで学習できる長さを返す。
            for iter_ in tqdm(range(len(self.train_dataloader))):
                # パラメータをtrainableにするため、feature_combinerをtrainモードにする。
                self.feature_combiner.train()

                # trainデータをサンプルする。
                x, y = self._sample_xy(data_type="train")

                # feature_combinerに特徴量を入力し、sentiment scoreを取得する。
                preds = self.feature_combiner(x=x)

                # sentiment scoreとラベルとのロスを計算する。
                train_loss = self.criterion(preds, y.view(-1, 1))

                # 計算されたロスは、後ほどepochごとのdisplayに使用するため、storeしておく。
                train_losses.append(train_loss.detach().cpu())

                # lossから、gradientを逆伝播させ、パラメータをupdateする。
                self._update_params(loss=train_loss)

                # validation用のロースを計算する。
                # 毎回計算を行うとコストがかかってくるので、iter_毎5回ごとに計算を行う。
                if iter_ % 5 == 0:

                    # 学習を行わないため、feature_combinerをevalモードにしておく。
                    # evalモードでは、dropoutの影響を受けない。
                    self.feature_combiner.eval()

                    # 各パラメータごとのgradientを計算するとリソースが高まる。
                    # evaluationの時には、gradient情報を持たせないことで、メモリーの節約に繋がる。
                    with torch.no_grad():
                        # validationデータをサンプルする
                        x, y = self._sample_xy(data_type="val")

                        # feature_combinerに特徴量を入力し、sentiment scoreを取得する。
                        preds = self.feature_combiner(x=x)

                        # sentiment scoreとラベルとのロスを計算する。
                        test_loss = self.criterion(preds, y.view(-1, 1))

                        # 計算されたロスは、後ほどepochごとのdisplayに使用するため、storeしておく。
                        test_losses.append(test_loss.detach().cpu())

            # 毎epoch終了後、平均のロスをプリントする。
            print(
                f"epoch: {epoch}, train_loss: {np.mean(train_losses):.4f}, val_loss: {np.mean(test_losses):.4f}"
            )

            # 毎epoch終了後、モデルのパラメータをstoreする。
            self._save_model(epoch=epoch)

    # 特徴量から、合成特徴量を抽出するロジック
    def combine_features(self, features):
        # 学習を行わないため、feature_combinerをevalモードにしておく。
        self.feature_combiner.eval()

        # gradient情報を持たせないことで、メモリーの節約する。
        with torch.no_grad():

            # 特徴量をfeature_combinerのextract_feature関数に入力し、出力層手前の特徴量を抽出する。
            # 抽出するとき、tensorをcpu上に落とし、np.ndarray形式に変換する。
            return (
                self.feature_combiner.extract_feature(
                    x=torch.tensor(features, dtype=torch.float).to(self.device)
                )
                .cpu()
                .numpy()
            )

    # 特徴量から、翌週のsentimentを予測するロジック
    def predict_sentiment(self, features):
        # 学習を行わないため、feature_combinerをevalモードにしておく。
        self.feature_combiner.eval()

        # gradient情報を持たせないことで、メモリーの節約する。
        with torch.no_grad():

            # 特徴量をfeature_combinerに入力し、sentiment scoreを抽出する。
            # 抽出するとき、tensorをcpu上に落とし、np.ndarray形式に変換する。
            return (
                self.feature_combiner(x=torch.tensor(features, dtype=torch.float).to(self.device))
                .cpu()
                .numpy()
            )

    # weeklyグループされた特徴量を入力に、合成特徴量もしくは、sentiment scoreを抽出するロジック
    def generate_by_weekly_features(
        self, weekly_features, generate_target, max_sequence_length
    ):
        assert generate_target in ("features", "sentiment")
        generate_func = getattr(
            self,
            {"features": "combine_features", "sentiment": "predict_sentiment"}[
                generate_target
            ],
        )

        # グループごとに特徴量もしくは、sentiment scoreを抽出し、最終的に重ねて返すため、リストを作成する。
        outputs = []

        # ユニークな週indexを取得する。
        weeks = sorted(weekly_features.index.get_level_values(0).unique())

        for week in tqdm(weeks):
            # 各週ごとの特徴量を取得し、直近から、max_sequence_length分切る。
            features = weekly_features.xs(week, axis=0, level=0)[-max_sequence_length:]

            # 特徴量をモデルに入力し、合成特徴量もしくは、sentiment scoreを抽出し、outputsにappendする。
            # np.expand_dims(features, axis=0)を用いる理由は、特徴量合成機の入力期待値は、dimention0がbatchであるが、
            # featuresは、[1000, 768]の次元をもち、これらをunsqueezeし、[1, 1000, 768]に変換する必要がある。
            outputs.append(generate_func(features=np.expand_dims(features, axis=0)))

        # outputsを重ね、indexの情報とともにpd.DataFrame形式として返す。
        return pd.DataFrame(np.concatenate(outputs, axis=0), index=weeks)

### 1.7.10. 特徴量合成モデルの学習及び特徴量合成
ここでは、実際作成したコードを元に、特徴量合成機の学習を行う。さらに、学習されたモデルを用いて、特徴量、sentiment scoreを抽出してみる。
モデル学習において、過学習防止のため、二つのランダム性を与えた。ランダム性を固定し、再現可能にするため、seed若しくはrandom_stateを固定する方法を取ることができる。しかし、実装においては二つのランダム性を同時に左右しないといけないことから、コードの難易度が少々高まってしまうため、本チュートリアルでは省くとする。学習の実行毎に類似であるが少し異なる結果となり得ることに留意しよう。また、推論ではランダム性の影響はない。

In [ ]:
# feature_combiner_handlerを定義する。そのとき、feature_combiner_paramsに{"input_size": 768, "hidden_size": 128}を渡しているが、
# これは、BERTから抽出した特徴量のサイズが768、LSTMに入力する次元が、[sequence, input_size]を持つため、"input_size"に768を与え、
# LSTMが持つ内部状態のパラメータの次元においては、適切な値をしておく。ここでは、128次元と設置しているが、過学習の恐れが高いとき、また、パラメータを減らしても十分学習受容量がある時には、より低い値をセットしよう。
feature_combiner_handler = FeatureCombinerHandler(feature_combiner_params={"input_size": 768, "hidden_size": 128}, store_dir=f'{CONFIG["base_path"]}/test')

In [ ]:
# 続き、学習用と、validation用のデータをビルドする必要がある。
# 一般的に、データは、学習に用いられる学習データと、パラメータなどの調整のため用いられるvalidationデータ、学習後のモデルを評価するためのtestデータに分けるTrain-Test- Validation分割を取ることが多いが、
# 本チュートリアルでは、データ数がかなり少ないため、Train-Test分割を行い、validation　lossの表示時にもtestデータを用いて行う。
# 上記で作成したbuild_weekly_featuresとbuild_weekly_labelsを用いて、データセットを生成する。
boundary_week = (2020, 26)
weekly_features = SentimentGenerator.build_weekly_features(features, boundary_week)
weekly_labels = SentimentGenerator.build_weekly_labels(stock_price, boundary_week)

In [ ]:
# 学習を行う前に、データのサンプル及び、batch処理を行ってくれるdataloaderをビルドする必要がある。

# train dataloaderをsetする。
# このとき、batch_sizeを4にすることで、4つのデータを並列に学習し、
# num_workersを2にすることでdataloaderはcpu 2coreを用いて、並列的にロードされる。
# max_sequence_lengthを1000にすることで、学習中には1000個のsequenceをmaxとして入力する。
feature_combiner_handler.set_train_dataloader(
    dataloader_params={
        "batch_size": 4,
        "num_workers": 1,
    },
    weekly_features=weekly_features['train'],
    weekly_labels=weekly_labels['train'], 
    max_sequence_length=1000
)

# validation dataloaderをsetする。
feature_combiner_handler.set_val_dataloader(
    dataloader_params={
        "batch_size": 4,
        "num_workers": 1,
    },
    weekly_features=weekly_features['test'],
    weekly_labels=weekly_labels['test'], 
    max_sequence_length=1000
)

In [ ]:
# 学習を行う。
# n_epochは全てのデータを一度用いた学習回数を表し、ここでは、テストであるため1と設定する。
feature_combiner_handler.train(n_epoch=1)

In [ ]:
# 学習後、特徴量抽出機から、特徴量やsentiment scoreを抽出することができる。
# 今回は、一度学習されたモデルをロードし、特徴量とsentiment scoreを抽出する。

# 上で定義した、feature_combiner_handlerを同様に定義すると、check_pointを探し、モデルがロードされる。
feature_combiner_handler = FeatureCombinerHandler(feature_combiner_params={"input_size": 768, "hidden_size": 128}, store_dir=f'{CONFIG["base_path"]}/test')

In [ ]:
# sentiment scoreは以下のように取得できる
# max_sequence_lengthは学習時と同様に、直近から利用する特徴量の最大の数を決めることができるが、
# 評価時には、十分長い(全部かほぼ全部)の特徴量を合成するため、10000を与えている。
sentiment_score = feature_combiner_handler.generate_by_weekly_features(weekly_features=weekly_features['test'], generate_target='sentiment', max_sequence_length=10000)
display(sentiment_score.head(3))
display(sentiment_score.tail(3))

In [ ]:
# 合成特徴量は以下のように取得できる
# max_sequence_lengthは学習時と同様に、直近から利用する特徴量の最大の数を決めることができるが、
# 評価時には、十分長い(全部かほぼ全部)の特徴量を合成するため、10000を与えている。
combined_features = feature_combiner_handler.generate_by_weekly_features(weekly_features=weekly_features['test'], generate_target='features', max_sequence_length=10000)
display(combined_features.head(3))
display(combined_features.tail(3))

In [ ]:
# これらのpd.Dataframeをstoreし、ロードしてみる。
# dataframeのstore時には、csv, pickle, parquet, h5など、さまざまな方法が存在する。
# 本章では扱いやすいpickleでのstoreを説明する。

# 以下のコマンドよりpickle形式で、storeすることができる。
combined_features.to_pickle(f'{CONFIG["base_path"]}/test/test.pkl')

In [ ]:
# 以下のコマンドよりファイルをロードすることができる。
combined_features = pd.read_pickle(f'{CONFIG["base_path"]}/test/test.pkl')

display(combined_features.head(3))
display(combined_features.tail(3))

In [ ]:
# 上で作成したFeatureCombinerHandlerは、本番環境において特徴量の合成時にも用いられる。SentimentGeneratorに、インスタンスとしてビルドしておこう。
SentimentGenerator.headline_feature_combiner_handler = FeatureCombinerHandler(feature_combiner_params={"input_size": 768, "hidden_size": 128}, store_dir=f'{CONFIG["base_path"]}/headline_features')
SentimentGenerator.keywords_feature_combiner_handler = FeatureCombinerHandler(feature_combiner_params={"input_size": 768, "hidden_size": 128}, store_dir=f'{CONFIG["base_path"]}/keywords_features')

In [ ]:
# 上記のコードをまとめ、headlineとkeywordsそれぞれにおいて、特徴量合成機を学習し、特徴量を抽出するコードを作成する。
boundary_week = (2020, 26)
for features, feature_type in [(headline_features, 'headline_features'), (keywords_features, 'keywords_features')]:
    # feature_typeに合致するfeature_combiner_handlerをSentimentGeneratorから取得する。
    feature_combiner_handler = {
        'headline_features': SentimentGenerator.headline_feature_combiner_handler,
        'keywords_features': SentimentGenerator.keywords_feature_combiner_handler,
    }[feature_type]
    
    # 学習及び、validationに用いる、データをビルドする
    weekly_features = SentimentGenerator.build_weekly_features(features, boundary_week)
    weekly_labels = SentimentGenerator.build_weekly_labels(stock_price, boundary_week)

    # train dataloaderをsetする。
    # このとき、batch_sizeを4にすることで、4つのデータを並列に学習し、
    # num_workersを2にすることでdataloaderはcpu 2coreを用いて、並列的にロードされる。
    feature_combiner_handler.set_train_dataloader(
        dataloader_params={
            "batch_size": 4,
            "num_workers": 1,
        },
        weekly_features=weekly_features['train'],
        weekly_labels=weekly_labels['train'], 
        max_sequence_length=1000
    )

    # validation dataloaderをsetする。
    feature_combiner_handler.set_val_dataloader(
        dataloader_params={
            "batch_size": 4,
            "num_workers": 1,
        },
        weekly_features=weekly_features['test'],
        weekly_labels=weekly_labels['test'], 
        max_sequence_length=1000
    )

    # 学習
    feature_combiner_handler.train(n_epoch=20)

    # 特徴量及びsentiment scoreを抽出し、pickleとしてstoreする。
    feature_combiner_handler.generate_by_weekly_features(weekly_features=weekly_features['test'], generate_target='sentiment', max_sequence_length=10000).to_pickle(os.path.join(f'{CONFIG["base_path"]}/{feature_type}', 'LSTM_sentiment.pkl'))
    feature_combiner_handler.generate_by_weekly_features(weekly_features=weekly_features['test'], generate_target='features', max_sequence_length=10000).to_pickle(os.path.join(f'{CONFIG["base_path"]}/{feature_type}', 'LSTM_features.pkl'))

### 1.7.11. 本番提出用のクラス作成

In [ ]:
# ここまでの、ニュースのデータをロード、前処理、BERT特徴量、LSTMによる特徴量合成までの一連の処理を
# `generate_lstm_features`関数として、SentimentGeneratorクラスに追加する。

@classmethod
def generate_lstm_features(
    cls,
    article_path,
    start_dt=None,
    boundary_week=(2020, 26),
    target_feature_types=None,
):
    # target_feature_typesが指定されなかったらデフォルト値設定
    dfault_target_feature_types = [
        "headline",
        "keywords",
    ]
    if target_feature_types is None:
        target_feature_types = dfault_target_feature_types

    # feature typeが想定通りであることを確認
    assert set(target_feature_types).issubset(dfault_target_feature_types)

    # ニュースデータをロードする。
    articles = cls.load_articles(start_dt=start_dt, path=article_path)

    # 前処理を行う。
    articles = cls.normalize_articles(articles)
    articles = cls.handle_punctuations_in_articles(articles)
    articles = cls.drop_remove_list_words(articles)

    # headlineとkeywordsの特徴量をdict型で返す。
    lstm_features = {}

    for feature_type in target_feature_types:
        # コーパス全体のBERT特徴量を抽出する。
        features = cls.generate_features_by_texts(texts=articles[feature_type])

        # feature_typeに合致するfeature_combiner_handlerをclsから取得する。
        feature_combiner_handler = {
            "headline": cls.headline_feature_combiner_handler,
            "keywords": cls.keywords_feature_combiner_handler,
        }[feature_type]

        # 特徴量を週毎のグループ化する。
        weekly_features = cls.build_weekly_features(features, boundary_week)["test"]

        # Sentiment scoreを抽出する。
        lstm_features[
            f"{feature_type}_features"
        ] = feature_combiner_handler.generate_by_weekly_features(
            weekly_features=weekly_features,
            generate_target="sentiment",
            max_sequence_length=10000,
        )

    return lstm_features

In [ ]:
# ここまでSentimentGeneratorに追加したclassmethodをまとめ、SentimentGeneratorクラスを仕上げる。

class SentimentGenerator(object):
    article_columns = None
    device = None
    feature_extractor = None
    headline_feature_combiner_handler = None
    keywords_feature_combiner_handler = None
    punctuation_replace_dict = None
    punctuation_remove_list = None

    @classmethod
    def initialize(cls, base_dir="../model"):
        # 使用するcolumnをセットする。
        cls.article_columns = ["publish_datetime", "headline", "keywords"]

        # BERT特徴量抽出機をセットする。
        cls._set_device()
        cls._build_feature_extractor()
        cls._build_tokenizer()

        # LSTM特徴量合成機をセットする。
        cls.headline_feature_combiner_handler = FeatureCombinerHandler(
            feature_combiner_params={"input_size": 768, "hidden_size": 128},
            store_dir=f"{base_dir}/headline_features",
        )
        cls.keywords_feature_combiner_handler = FeatureCombinerHandler(
            feature_combiner_params={"input_size": 768, "hidden_size": 128},
            store_dir=f"{base_dir}/keywords_features",
        )

        # 置換すべき記号のdictionaryを作成する。
        JISx0208_replace_dict = {
            "髙": "高",
            "﨑": "崎",
            "濵": "浜",
            "賴": "頼",
            "瀨": "瀬",
            "德": "徳",
            "蓜": "配",
            "昻": "昂",
            "桒": "桑",
            "栁": "柳",
            "犾": "犹",
            "琪": "棋",
            "裵": "裴",
            "魲": "鱸",
            "羽": "羽",
            "焏": "丞",
            "祥": "祥",
            "曻": "昇",
            "敎": "教",
            "澈": "徹",
            "曺": "曹",
            "黑": "黒",
            "塚": "塚",
            "閒": "間",
            "彅": "薙",
            "匤": "匡",
            "冝": "宜",
            "埇": "甬",
            "鮏": "鮭",
            "伹": "但",
            "杦": "杉",
            "罇": "樽",
            "柀": "披",
            "﨤": "返",
            "寬": "寛",
            "神": "神",
            "福": "福",
            "礼": "礼",
            "贒": "賢",
            "逸": "逸",
            "隆": "隆",
            "靑": "青",
            "飯": "飯",
            "飼": "飼",
            "緖": "緒",
            "埈": "峻",
        }

        cls.punctuation_replace_dict = {
            **JISx0208_replace_dict,
            "《": "〈",
            "》": "〉",
            "『": "「",
            "』": "」",
            "“": '"',
            "!!": "!",
            "〔": "[",
            "〕": "]",
            "χ": "x",
        }

        # 取り除く記号リスト。
        cls.punctuation_remove_list = [
            "|",
            "■",
            "◆",
            "●",
            "★",
            "☆",
            "♪",
            "〃",
            "△",
            "○",
            "□",
        ]

    @classmethod
    def _set_device(cls):
        # 使用可能なgpuがある場合、そちらを利用し特徴量抽出を行う
        if torch.cuda.device_count() >= 1:
            cls.device = "cuda"
            print("[+] Set Device: GPU")
        else:
            cls.device = "cpu"
            print("[+] Set Device: CPU")

    @classmethod
    def _build_feature_extractor(cls):
        # 特徴量抽出のため事前学習済みBERTモデルを用いる。
        # ここでは、"cl-tohoku/bert-base-japanese-whole-word-masking"モデルを使用しているが、異なる日本語BERTモデルを用いても良い。
        cls.feature_extractor = transformers.BertModel.from_pretrained(
            "cl-tohoku/bert-base-japanese-whole-word-masking",
            return_dict=True,
            output_hidden_states=True,
        )

        # 使用するdeviceを指定
        cls.feature_extractor = cls.feature_extractor.to(cls.device)

        # 今回、学習は行わない。特徴量抽出のためなので、評価モードにセットする。
        cls.feature_extractor.eval()

        print("[+] Built feature extractor")

    @classmethod
    def _build_tokenizer(cls):
        # BERTモデルの入力とするコーパスはそのBERTモデルが学習された時と同様の前処理を行う必要がある。
        # 今回使用する"cl-tohoku/bert-base-japanese-whole-word-masking"モデルは、mecab-ipadic-NEologdによりトークナイズされ、その後Wordpiece subword encoderよりsubword化している。
        # Subwordとは形態素の類似な概念として、単語をより小さい意味のある単位に変換したものである。
        # transformersのBertJapaneseTokenizerは、その事前学習モデルの学習時と同様の前処理を簡単に使用することができる。
        # この章ではBertJapaneseTokenizerを利用し、トークナイズ及びsubword化を行う。
        cls.bert_tokenizer = BertJapaneseTokenizer.from_pretrained(
            "cl-tohoku/bert-base-japanese-whole-word-masking"
        )
        print("[+] Built bert tokenizer")

    @classmethod
    def load_articles(cls, path, start_dt=None, end_dt=None):
        # csvをロードする
        # headline、keywordsをcolumnとして使用。publish_datetimeをindexとして使用。
        articles = pd.read_csv(path)[cls.article_columns].set_index("publish_datetime")

        # str形式のdatetimeをpd.Timestamp形式に変換
        articles.index = pd.to_datetime(articles.index)

        # NaN値を取り除く
        articles = articles.dropna()

        # 必要な場合、使用するデータの範囲を指定する
        return articles[start_dt:end_dt]

    @classmethod
    def normalize_articles(cls, articles):
        articles = articles.copy()

        # 欠損値を取り除く
        articles = articles.dropna()

        for column in articles.columns:
            # スペース(全角スペースを含む)はneologdn正規化時に全て除去される。
            # ここでは、スペースの情報が失われないように、スペースを全て改行に書き換え、正規化後スペースに再変換する。
            articles[column] = articles[column].apply(lambda x: "\n".join(x.split()))

            # neologdnを使って正規化を行う。
            articles[column] = articles[column].apply(lambda x: neologdn.normalize(x))

            # 改行をスペースに置換する。
            articles[column] = articles[column].str.replace("\n", " ")

        return articles

    @classmethod
    def handle_punctuations_in_articles(cls, articles):
        articles = articles.copy()

        for column in articles.columns:
            # punctuation_remove_listに含まれる記号を除去する
            articles[column] = articles[column].str.replace(
                fr"[{''.join(cls.punctuation_remove_list)}]", ""
            )

            # punctuation_replace_dictに含まれる記号を置換する
            for replace_base, replace_target in cls.punctuation_replace_dict.items():
                articles[column] = articles[column].str.replace(
                    replace_base, replace_target
                )

            # unicode正規化を行う
            articles[column] = articles[column].apply(
                lambda x: unicodedata.normalize("NFKC", x)
            )

        return articles

    @classmethod
    def drop_remove_list_words(cls, articles, remove_list_words=["人事"]):
        articles = articles.copy()

        for remove_list_word in remove_list_words:
            # headlineもしくは、keywordsどちらかでremove_list_wordを含むニュース記事のindexマスクを作成。
            drop_mask = articles["headline"].str.contains(remove_list_word) | articles[
                "keywords"
            ].str.contains(remove_list_word)

            # remove_list_wordを含まないニュースだけに精製する。
            articles = articles[~drop_mask]

        return articles

    @classmethod
    def build_inputs(cls, texts, max_length=512):
        input_ids = []
        token_type_ids = []
        attention_mask = []
        for text in texts:
            encoded = cls.bert_tokenizer.encode_plus(
                text,
                None,
                add_special_tokens=True,
                max_length=max_length,
                padding="max_length",
                return_token_type_ids=True,
                truncation=True,
            )

            input_ids.append(encoded["input_ids"])
            token_type_ids.append(encoded["token_type_ids"])
            attention_mask.append(encoded["attention_mask"])

        # torchモデルに入力するためにはtensor形式に変え、deviceを指定する必要がある。
        input_ids = torch.tensor(input_ids, dtype=torch.long).to(cls.device)
        token_type_ids = torch.tensor(token_type_ids, dtype=torch.long).to(cls.device)
        attention_mask = torch.tensor(attention_mask, dtype=torch.long).to(cls.device)

        return input_ids, token_type_ids, attention_mask

    @classmethod
    def generate_features(cls, input_ids, token_type_ids, attention_mask):
        output = cls.feature_extractor(
            input_ids=input_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask,
        )
        features = output["hidden_states"][-2].mean(dim=1).cpu().detach().numpy()

        return features

    @classmethod
    def generate_features_by_texts(cls, texts, batch_size=2, max_length=512):
        n_batch = math.ceil(len(texts) / batch_size)

        features = []
        for idx in tqdm(range(n_batch)):
            input_ids, token_type_ids, attention_mask = cls.build_inputs(
                texts=texts[batch_size * idx : batch_size * (idx + 1)],
                max_length=max_length,
            )

            features.append(
                cls.generate_features(
                    input_ids=input_ids,
                    token_type_ids=token_type_ids,
                    attention_mask=attention_mask,
                )
            )

        features = np.concatenate(features, axis=0)

        # 抽出した特徴量はnp.ndarray形式となっており、これらは、日付の情報を失っているため、pd.DataFrame形式に変換する。
        return pd.DataFrame(features, index=texts.index)

    @classmethod
    def _build_weekly_group(cls, df):
        # index情報から、(year, week)の情報を得る。
        return pd.Series(list(zip(df.index.year, df.index.week)), index=df.index)

    @classmethod
    def build_weekly_features(cls, features, boundary_week):
        assert isinstance(boundary_week, tuple)

        weekly_group = cls._build_weekly_group(df=features)
        features = features.groupby(weekly_group).apply(lambda x: x[:])

        train_features = features[features.index.get_level_values(0) <= boundary_week]
        test_features = features[features.index.get_level_values(0) > boundary_week]

        return {"train": train_features, "test": test_features}

    @classmethod
    def generate_lstm_features(
        cls,
        article_path,
        start_dt=None,
        boundary_week=(2020, 26),
        target_feature_types=None,
    ):
        # target_feature_typesが指定されなかったらデフォルト値設定
        dfault_target_feature_types = [
            "headline",
            "keywords",
        ]
        if target_feature_types is None:
            target_feature_types = dfault_target_feature_types
        # feature typeが想定通りであることを確認
        assert set(target_feature_types).issubset(dfault_target_feature_types)

        # ニュースデータをロードする。
        articles = cls.load_articles(start_dt=start_dt, path=article_path)

        # 前処理を行う。
        articles = cls.normalize_articles(articles)
        articles = cls.handle_punctuations_in_articles(articles)
        articles = cls.drop_remove_list_words(articles)

        # headlineとkeywordsの特徴量をdict型で返す。
        lstm_features = {}

        for feature_type in target_feature_types:
            # コーパス全体のBERT特徴量を抽出する。
            features = cls.generate_features_by_texts(texts=articles[feature_type])

            # feature_typeに合致するfeature_combiner_handlerをclsから取得する。
            feature_combiner_handler = {
                "headline": cls.headline_feature_combiner_handler,
                "keywords": cls.keywords_feature_combiner_handler,
            }[feature_type]

            # 特徴量を週毎のグループ化する。
            weekly_features = cls.build_weekly_features(features, boundary_week)["test"]

            # Sentiment scoreを抽出する。
            lstm_features[
                f"{feature_type}_features"
            ] = feature_combiner_handler.generate_by_weekly_features(
                weekly_features=weekly_features,
                generate_target="sentiment",
                max_sequence_length=10000,
            )

        return lstm_features

## 1.7.12. 合成特徴量の解析
今回前章において、作成したlstmモデルからfeaturesとsentimentを抽出した。sentimentはfeaturesから線形次元圧縮されたものであり、featuresのほうがより高次元として、高い情報を保持しているが、使用時の容易性などより続くチュートリアルにおいては、sentimentのみを用いてモデリングを行う。本章では、sentiment合成特徴量がどのような性質を獲得しているかを確認するため、学習のオブジェクトとなるマーケットのforward returnとの関係性を解析する。

### 1.7.13.  合成特徴量のロード

In [ ]:
# 前章で抽出したlstm_sentimentをロードする。
# コラムは、次元順に付与されたidを表し、sentimentの場合は１次元のみのデータとなっているため、0でインデクシングを行って以下、解析ではpd.series形式として扱う。
headline_features = pd.read_pickle(f'{CONFIG["base_path"]}/headline_features/LSTM_sentiment.pkl')[0].rename('features')
keywords_features = pd.read_pickle(f'{CONFIG["base_path"]}/keywords_features/LSTM_sentiment.pkl')[0].rename('features')

display(headline_features.head())
display(keywords_features.head())

### 1.7.14. ラベルをビルド

In [ ]:
# boundary_weekを学習時境界と同様に設定し、weekly_fwd_returnsをビルドする。
weekly_group = SentimentGenerator._build_weekly_group(df=stock_price)
weekly_returns = stock_price.groupby(weekly_group).apply(_compute_weekly_return)
weekly_fwd_returns = weekly_returns.shift(-1).rename('weekly_fwd_returns')

# 特徴量の期間と同様の期間のデータのみを使用する。
weekly_fwd_returns = weekly_fwd_returns.reindex(headline_features.index).dropna()

display(weekly_fwd_returns)

### 1.7.15. 相関係数確認

In [ ]:
# 二つのpd.Seriesをconcatenateする。
# indexの違いがあるため、片方だけ存在するデータはドロップする。
df = pd.concat([headline_features, weekly_fwd_returns], axis=1, sort=True).dropna()

# 二つのコラムのシークエンス間の相関は、以下のように取得できる。
display(df.corr()[df.columns[0]][df.columns[-1]])

In [ ]:
# corr関数にmethodを指定して、pearson及びspearman相関係数を表示する。
def display_corr(df):
    display(pd.Series(
        {
            "pearson": df.corr(method='pearson')[df.columns[0]][df.columns[-1]],
            "spearman": df.corr(method='spearman')[df.columns[0]][df.columns[-1]]
        }
    ))
    
display_corr(df)

In [ ]:
# 上記の関数をheadline, keywords両方に適用に、ラベルとの相関を表示する。
for features, feature_type in [(headline_features, 'headline_features'), (keywords_features, 'keywords_features')]:
    display_markdown(f'#### feature_type: {feature_type}', raw=True)
    df = pd.concat([features, weekly_fwd_returns], axis=1, sort=True).dropna()
    display_corr(df)

### 1.7.16. 可視化

In [ ]:
# 回帰を行うため、xとyとなるコラムを設定する。
x_column = 'features'
y_column = 'weekly_fwd_returns'

# stats.linregressを用いて、単回帰直線の係数とバイアスを取得する。
df = pd.concat([headline_features, weekly_fwd_returns], axis=1, sort=True).dropna()

coef, bias, _, _, _ = stats.linregress(x=df[x_column], y=df[y_column])
print(f'coef: {coef:.4f}, bias: {bias:.4f}')

In [ ]:
def display_regplot(df, x_column='features', y_column='weekly_fwd_returns'):
    # stats.linregressを用いて、単回帰直線の係数とバイアスを取得する。
    coef, bias, _, _, _ = stats.linregress(x=df[x_column], y=df[y_column])

    # seabornのregplotを用いて、単回帰直線及び、scatter sampleを表示する。
    _, ax = plt.subplots(1, 1, figsize=(6, 4))
    sns.regplot(
        x=x_column,
        y=y_column,
        data=df,
        ax=ax,
        line_kws={
            "label": "y={0:.4f}x+{1:.4f}".format(coef, bias), # 取得した係数とバイアスを用いて単項式を表示する。
        },
    )
    plt.legend()
    plt.show()

for features, feature_type in [(headline_features, 'headline_features'), (keywords_features, 'keywords_features')]:
    display_markdown(f'#### feature_type: {feature_type}', raw=True)
    df = pd.concat([features, weekly_fwd_returns], axis=1, sort=True).dropna()
    display_regplot(df=df)

In [ ]:
# barplotより、特徴量とラベル間での相互関係を確認する。
# 二つのデータを一つの軸上で単純にプロットすると値のノルムや平均、分散の違いから、相互的な動きを確認しにくい。
df.plot(kind='bar', figsize=(16, 3))

In [ ]:
# ここでは、手元にあるデータを平均と分散を用いてノーマライズしたzscoreを使って相互関係可視化してみよう。
def normalize(df):
    # zscore normalizeする。
    return pd.DataFrame(stats.zscore(df), index=df.index, columns=[f'Z({column})' for column in df.columns])


def display_bar_plot(df):
    # zscore normalizeしたデータを用いてbarplotする。
    normalize(df).plot(kind='bar', figsize=(16, 3))
    plt.show()
    

for features, feature_type in [(headline_features, 'headline_features'), (keywords_features, 'keywords_features')]:
    display_markdown(f'#### feature_type: {feature_type}', raw=True)
    df = pd.concat([features, weekly_fwd_returns], axis=1, sort=True).dropna()
    display_bar_plot(df=df)